## Lab 1. Object detection

This notebook is intended to be run along with the README.md file.

## Preparation

In [ ]:
# Import libraries

import sys
import os
import json
import glob
import tarfile

import boto3
import sagemaker
import IPython
import gluoncv

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

In [ ]:
# Initialize variables and configurations

boto3_session = boto3.session.Session()
sm_session = sagemaker.Session()

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3_session.region_name
s3_bucket = sm_session.default_bucket()
sm_role = sagemaker.get_execution_role()

print( "account_id :", account_id )
print( "region :", region )
print( "s3_bucket :", s3_bucket )
print( "sm_role :", sm_role )

## "Hello World!" application

In [ ]:
app_name = "lab1"

!panorama-cli init-project --name {app_name}

In [ ]:
code_package_name = f"{app_name}_code"
code_package_version = "1.0"
source_filename = f"./{app_name}/packages/{account_id}-{code_package_name}-{code_package_version}/src/app.py"

!cd {app_name} && panorama-cli create-package --type Container --name {code_package_name} --version {code_package_version}

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> Manually create a one-line python source code referring to README.md file.</div>

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> Manually edit the code node descriptor file, and specify the entry point of the application code, referring to README.md file.</div>

In [ ]:
!cd {app_name} && panorama-cli build-container --container-asset-name code --package-path packages/{account_id}-{code_package_name}-{code_package_version}

In [ ]:
%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--py-file {source_filename}

## Video pass-through application

In [ ]:
camera_node_name = f"{app_name}_camera"

!cd {app_name} && panorama-cli add-panorama-package --type camera --name {camera_node_name}

In [ ]:
data_sink_node_name = f"{app_name}_data_sink"

!cd {app_name} && panorama-cli add-panorama-package --type data_sink --name {data_sink_node_name}

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> Manually edit the graph.json file, and connect code node, camera node, and data sink node, referring to README.md file.</div>

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> Manually edit the app.py referring to README.md file.</div>

In [ ]:
video_filepath = "../../videos/TownCentreXVID.avi"

%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--py-file {source_filename} \
\
--camera-node-name lab1_camera \
--video-file {video_filepath} \
--video-start 0 \
--video-stop 10 \
--video-step 1 \
\
--output-screenshots ./screenshots/%Y%m%d_%H%M%S

In [ ]:
# View latest screenshot image

latest_screenshot_dirname = sorted( glob.glob( "./screenshots/*" ) )[-1]
screenshot_filename = sorted( glob.glob( f"{latest_screenshot_dirname}/*.png" ) )[-1]

print(screenshot_filename)
IPython.display.Image( filename = screenshot_filename )

## People detection application

In [ ]:
def export_model_and_create_targz( prefix, name, model ):
    os.makedirs( prefix, exist_ok=True )
    gluoncv.utils.export_block( os.path.join( prefix, name ), model, preprocess=False, layout="CHW" )

    tar_gz_filename = f"{prefix}/{name}.tar.gz"
    with tarfile.open( tar_gz_filename, "w:gz" ) as tgz:
        tgz.add( f"{prefix}/{name}-symbol.json", f"{name}-symbol.json" )
        tgz.add( f"{prefix}/{name}-0000.params", f"{name}-0000.params" )
        
    print( f"Exported : {tar_gz_filename}" )
    
# Export object detection model. Reset the classes for human detection only.
people_detection_model = gluoncv.model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
people_detection_model.reset_class(["person"], reuse_weights=['person'])
export_model_and_create_targz( "models", "yolo3_mobilenet1.0_coco_person", people_detection_model )

In [ ]:
model_package_name = f"{app_name}_model"
model_package_version = "1.0"
people_detection_model_name = "people_detection_model"

!cd {app_name} && panorama-cli create-package --name {model_package_name} --type Model --version {code_package_version}

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> Manually edit the model descriptor file, and specify the name of ML framework ("MXNET"), input data name ("data") and input data shape ( [1, 3, 480, 600] ),  referring to README.md file.</div>


In [ ]:
!cd {app_name} && panorama-cli add-raw-model \
    --model-asset-name {people_detection_model_name} \
    --model-local-path ../models/yolo3_mobilenet1.0_coco_person.tar.gz \
    --descriptor-path packages/{account_id}-{model_package_name}-{model_package_version}/descriptor.json \
    --packages-path packages/{account_id}-{model_package_name}-{model_package_version}

In [ ]:
people_detection_model_data_shape = '{"data":[1,3,480,600]}'

%run ../common/test_utility/panorama_test_utility_compile.py \
\
--s3-model-location s3://{s3_bucket}/panorama-workshop/{app_name} \
\
--model-node-name {people_detection_model_name} \
--model-file-basename ./models/yolo3_mobilenet1.0_coco_person \
--model-data-shape '{people_detection_model_data_shape}' \
--model-framework MXNET

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> Manually edit the app.py referring to README.md file.</div>

In [ ]:
video_filepath = "../../videos/TownCentreXVID.avi"

%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--py-file {source_filename} \
\
--model-package-name {model_package_name} \
--model-node-name {people_detection_model_name} \
--model-file-basename ./models/yolo3_mobilenet1.0_coco_person \
\
--camera-node-name lab1_camera \
\
--video-file {video_filepath} \
--video-start 0 \
--video-stop 10 \
--video-step 1 \
\
--output-screenshots ./screenshots/%Y%m%d_%H%M%S

In [ ]:
# View latest screenshot image

latest_screenshot_dirname = sorted( glob.glob( "./screenshots/*" ) )[-1]
screenshot_filename = sorted( glob.glob( f"{latest_screenshot_dirname}/*.png" ) )[-1]

print(screenshot_filename)
IPython.display.Image( filename = screenshot_filename )

## Run the people detection application on real device

In [ ]:
!aws panorama list-devices

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> Manually edit the Docker file, and add some python libraries referring to README.md file.</div>

In [ ]:
!cd {app_name} && panorama-cli build-container --container-asset-name code --package-path packages/{account_id}-{code_package_name}-{code_package_version}

In [ ]:
!cd {app_name} && panorama-cli package-application

<div class="alert alert-block alert-warning"><b>Manual operation needed:</b> Deploy the application using the AWS Management Console referring to README.md file.
</div>

<div class="alert alert-block alert-warning"><b>Manual operation needed:</b> After confirming the execution of application, delete the application, referring to README.md file.
</div>